## Pre process
> all thats happening here is the some basic cleaning of notes and rewriting with a pipe delimiter so they can be read into spark in azure

> and the creation of the words of interest which are more predictive

In [1]:
pwd

'/home/ucsddse230/work/final_project/pre_azure'

In [ ]:
!pip install nltk


In [16]:
## note that ntlk is only being used on a small file to find disgnosis and procedures
from nltk import word_tokenize
import nltk
import pandas as pd
import string

In [ ]:
# if not downloaded
ntlk.download('punkt')

> link to code downloads
https://www.cms.gov/Medicare/Coding/ICD9ProviderDiagnosticCodes/codes.html

In [12]:
path_to_data='/home/ucsddse230/work/final_project/pre_azure'

# create list of idc9 codes 

In [13]:
def preprocess_text(df):
    # This function preprocesses the text by filling not a number and replacing new lines ('\n') and carriage returns ('\r')
    df.TEXT = df.TEXT.fillna(' ')
    df.TEXT = df.TEXT.str.replace('\n',' ')
    df.TEXT = df.TEXT.str.replace('\r',' ')
    return df



In [14]:
#load idc9 descriptions
notes_df = pd.read_table("{}/idc9_short.txt".format(path_to_data),header=None)
#collapse to string
dirty_idc9=" ".join(list(notes_df[0]))
#turn to pandas
dirty_df=pd.DataFrame([dirty_idc9],columns=['TEXT'])
#remove /n /r and fill na
cleener_idc9=preprocess_text(dirty_df)
#get values
idc9_text=cleener_idc9["TEXT"][0]
#create translate dictionary
punc_list = string.punctuation+'0123456789'
t = str.maketrans(dict.fromkeys(punc_list, " "))
#use translate dictionary
idc9_text = idc9_text.lower().translate(t)
idc9tokens = word_tokenize(idc9_text)

## Create CSV Filter of IDC Procedure and Diagnosis Words

In [23]:
pd.DataFrame(list(set(idc9tokens )), columns=['IDC_filter']).to_csv('IDC9_filter.csv', index=False)

In [9]:
print(list(set(idc9tokens )))

['nonrupt', 'ectodermal', 'anomal', 'prostatitis', 'thrombocytopen', 'unspcf', 'occupational', 'scleral', 'decrease', 'into', 'petit', 'uncnr', 'empyema', 'epispadias', 'aggressiv', 'nonsurg', 'antepart', 'correct', 'parasitic', 'afric', 'femoral', 'complicated', 'exos', 'prtoneum', 'abuse', 'meningismus', 'tricusp', 'oblit', 'histo', 'ac', 'pyeloureteritis', 'sphincter', 'hmrg', 'accommodative', 'mesh', 'persisting', 'sporotrichosis', 'dsct', 'perineum', 'pathol', 'curve', 'hep', 'pilaris', 'cortical', 'convalescence', 'intraut', 'intr', 'amnestc', 'midcervic', 'recurvatum', 'restrain', 'bursal', 'intervent', 'cryptococcal', 'palm', 'currnt', 'retracted', 'areata', 'fumes', 'failure', 'placen', 'mutism', 'ileum', 'involun', 'bronchitis', 'retrograde', 'convert', 'schizotypal', 'sublingual', 'kpsi', 'gs', 'hemoglobinuria', 'coxsackie', 'flaccid', 'hemolysis', 'secondry', 'subac', 'menstl', 'appl', 'disrup', 'expose', 'septicem', 'pro', 'br', 'engorgemnt', 'mandibular', 'explosn', 'magn

## preprocess notes
> ideally this is done in hive or some other ETL that can write to pipe delimited that spark has no problem reading
> for some reason in current state only pandas can import 

In [ ]:
notes_df = pd.read_csv("{}/NOTEEVENTS.csv".format(path_to_data))
##subset to used columns discharge summary
notes_filt_df=notes_df[['SUBJECT_ID','HADM_ID','CATEGORY','TEXT']][notes_df.CATEGORY=='Discharge summary']
#clean
notes_filt_df=preprocess_text(notes_filt_df )


In [ ]:
notes_filt_df.to_csv('notes_discharge_pd.csv',index=False, sep="|")

## Ready for upload into azure
```python
'notes_discharge_pd.csv' , 'ADMISSIONS.CSV'  and 'IDC9_filter.csv'
```

